In [1]:
import numpy as np
import pandas as pd

In [2]:
schedule = pd.read_csv("https://raw.githubusercontent.com/njj06135/Baseball_ChilliShrimp/master/data/schedule.csv", encoding = 'CP949')
pa_b = pd.read_csv("https://raw.githubusercontent.com/njj06135/Baseball_ChilliShrimp/master/data/batter_predict_pa.csv")
modeling_b = pd.read_csv("https://raw.githubusercontent.com/njj06135/Baseball_ChilliShrimp/master/data/batter_feature_selection.csv")

### batter

In [3]:
batter = modeling_b.loc[(modeling_b['year']==2020) & (modeling_b['month']>7)]

In [4]:
batter = batter[['T_ID', 'P_ID']].drop_duplicates()

In [5]:
pa_b['P_ID']=pa_b['P_ID'].astype('int')

In [6]:
batter_left= pd.merge(pa_b, batter, left_on='P_ID', right_on='P_ID', how='left')
batter_left

,year,month,P_ID,PA,T_ID
0,2020,10,65304,3.0,WO
1,2020,10,64012,10.0,KT
2,2020,10,68204,4.0,OB
3,2020,10,66203,6.0,OB
4,2020,10,50293,2.0,OB
...,...,...,...,...,...
222,2020,10,63435,4.0,SS
223,2020,10,69418,0.0,SS
224,2020,10,61457,0.0,SS
225,2020,10,66707,7.0,HH


In [7]:
batter_tidy = pd.read_csv("https://raw.githubusercontent.com/njj06135/Baseball_ChilliShrimp/master/data/batter_tidy.csv")

In [8]:
babip_total = batter_tidy.groupby(['P_ID']).sum()

In [9]:
babip_total['BABIP'] = (babip_total['HIT'] - babip_total['HR']) / (babip_total['AB'] - babip_total['KK'] - babip_total['HR'] + babip_total['SF'])

In [10]:
babip_total = babip_total[['BABIP']]

In [11]:
babip_total = babip_total.fillna(0)

In [12]:
babip_total

,BABIP
P_ID,
50054,0.250000
50066,0.277778
50150,0.434783
50165,0.331950
50202,0.240000
...,...
99445,0.000000
99563,0.000000
99606,0.362687


In [13]:
batter_2020 = batter_tidy.loc[batter_tidy.year == 2020, :]
batter_2020 = batter_2020.groupby(['P_ID']).sum()

In [14]:
batter_2020['BABIP_2020'] = (batter_2020['HIT'] - batter_2020['HR']) / (batter_2020['AB'] - batter_2020['KK'] - batter_2020['HR'] + batter_2020['SF'])

In [15]:
babip_2020 = batter_2020[['BABIP_2020']]

In [16]:
babip_total['P_ID'] = babip_total.index
babip_total = babip_total.reset_index(drop = True)

In [17]:
babip_2020['P_ID'] = babip_2020.index

C:\Users\dhxog\anaconda3\envs\my\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
babip_2020 = babip_2020.reset_index(drop = True)

In [19]:
batter_left= pd.merge(batter_left, babip_total, left_on='P_ID', right_on='P_ID', how='left')
batter_left= pd.merge(batter_left, babip_2020, left_on='P_ID', right_on='P_ID', how='left')


In [20]:
batter_left = batter_left.fillna(0)

In [21]:
batter_left['total_BABIP']=batter_left['BABIP_2020']-(batter_left['BABIP_2020']-batter_left['BABIP'])/5
batter_left

,year,month,P_ID,PA,T_ID,BABIP,BABIP_2020,total_BABIP
0,2020,10,65304,3.0,WO,0.375000,1.000000,0.875000
1,2020,10,64012,10.0,KT,0.297619,0.227273,0.241342
2,2020,10,68204,4.0,OB,0.315789,0.315789,0.315789
3,2020,10,66203,6.0,OB,0.285714,0.352941,0.339496
4,2020,10,50293,2.0,OB,0.266667,0.266667,0.266667
...,...,...,...,...,...,...,...,...
222,2020,10,63435,4.0,SS,0.166667,0.166667,0.166667
223,2020,10,69418,0.0,SS,0.299363,0.300699,0.300432
224,2020,10,61457,0.0,SS,0.000000,0.000000,0.000000
225,2020,10,66707,7.0,HH,0.272727,0.285714,0.283117


In [22]:
df1 = batter_tidy.copy()
df1.head()

,year,month,P_ID,T_ID,AB,HIT,H2,H3,HR,BB,...,VS_T_ID_KT,VS_T_ID_LG,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,HOME,AWAY
0,2016,4,60146,LG,0,0,0,0,0,0,...,2,0,3,3,0,3,2,3,10,12
1,2016,4,60263,NC,0,0,0,0,0,0,...,0,3,4,0,3,3,3,2,9,14
2,2016,4,60404,HH,12,2,0,0,0,0,...,0,4,3,3,6,0,2,3,12,11
3,2016,4,60456,SS,65,15,3,1,0,5,...,6,2,3,3,4,0,0,0,10,13
4,2016,4,60523,LT,91,22,4,1,0,7,...,3,3,0,4,0,3,3,3,14,11


In [23]:
df1 = df1.loc[df1['year']==2020]

In [24]:
df1

,year,month,P_ID,T_ID,AB,HIT,H2,H3,HR,BB,...,VS_T_ID_KT,VS_T_ID_LG,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,HOME,AWAY
6273,2020,5,50066,KT,16,4,2,0,0,1,...,0,3,3,3,2,0,3,3,12,11
6274,2020,5,50150,LG,0,0,0,0,0,0,...,3,0,0,2,3,3,3,2,11,11
6275,2020,5,50165,LG,80,30,5,0,10,10,...,3,0,0,2,3,3,3,2,11,11
6276,2020,5,50202,OB,14,3,0,0,0,1,...,2,3,6,3,0,3,3,0,11,12
6277,2020,5,50300,WO,35,4,0,0,1,1,...,3,2,3,3,0,3,3,0,12,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7419,2020,9,63435,SS,4,0,0,0,0,0,...,4,2,2,3,4,2,0,2,13,11
7420,2020,9,69418,SS,0,0,0,0,0,0,...,4,2,2,3,4,2,0,2,13,11
7421,2020,9,61457,SS,0,0,0,0,0,0,...,4,2,2,3,4,2,0,2,13,11
7422,2020,9,66707,HH,7,0,0,0,0,0,...,2,2,2,2,5,2,2,4,16,9


In [25]:
df1 = df1.groupby(['P_ID']).sum()
df1['inplay_ratio'] = (df1['AB']-df1['KK']-df1['HR']+df1['SF'])/df1['PA']

In [26]:
df1

,year,month,AB,HIT,H2,H3,HR,BB,HP,KK,...,VS_T_ID_LG,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,HOME,AWAY,inplay_ratio
P_ID,,,,,,,,,,,,,,,,,,,,,
50054,8080,30,65,13,3,0,0,7,1,13,...,8,9,12,10,12,10,10,45,47,0.712329
50066,6060,18,25,6,2,0,1,2,0,6,...,7,9,11,5,4,7,6,36,32,0.666667
50150,10100,35,29,10,2,0,0,1,0,6,...,0,12,11,15,14,13,14,56,61,0.766667
50165,10100,35,399,114,17,2,34,51,3,128,...,0,12,11,15,14,13,14,56,61,0.517167
50202,8080,26,28,6,0,0,0,1,0,4,...,13,14,14,0,10,12,6,47,46,0.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79456,10100,35,223,68,9,2,0,15,4,25,...,14,16,13,11,13,16,0,65,59,0.808000
79545,2020,6,0,0,0,0,0,0,0,0,...,3,0,1,0,0,3,3,10,13,NaN
79608,10100,35,371,106,26,0,6,32,7,44,...,12,0,10,14,12,13,16,56,60,0.778571


In [27]:
df1['P_ID'] = df1.index

In [28]:
df1 = df1.reset_index(drop = True)

In [29]:
df1

,year,month,AB,HIT,H2,H3,HR,BB,HP,KK,...,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,HOME,AWAY,inplay_ratio,P_ID
0,8080,30,65,13,3,0,0,7,1,13,...,9,12,10,12,10,10,45,47,0.712329,50054
1,6060,18,25,6,2,0,1,2,0,6,...,9,11,5,4,7,6,36,32,0.666667,50066
2,10100,35,29,10,2,0,0,1,0,6,...,12,11,15,14,13,14,56,61,0.766667,50150
3,10100,35,399,114,17,2,34,51,3,128,...,12,11,15,14,13,14,56,61,0.517167,50165
4,8080,26,28,6,0,0,0,1,0,4,...,14,14,0,10,12,6,47,46,0.833333,50202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,10100,35,223,68,9,2,0,15,4,25,...,16,13,11,13,16,0,65,59,0.808000,79456
350,2020,6,0,0,0,0,0,0,0,0,...,0,1,0,0,3,3,10,13,NaN,79545
351,10100,35,371,106,26,0,6,32,7,44,...,0,10,14,12,13,16,56,60,0.778571,79608
352,4040,11,18,3,1,0,0,3,1,6,...,6,9,2,6,3,6,23,24,0.600000,79705


In [30]:
df1.isna().sum()

year             0
month            0
AB               0
HIT              0
H2               0
H3               0
HR               0
BB               0
HP               0
KK               0
GD               0
PA               0
SB               0
CS               0
SF               0
BAT_ORDER        0
stadium_MH       0
stadium_MS       0
stadium_JS       0
stadium_SW       0
stadium_DJ       0
stadium_DG       0
stadium_GJ       0
stadium_GC       0
stadium_SJ       0
stadium_PH       0
stadium_US       0
stadium_CJ       0
stadium_CW       0
VS_T_ID_HH       0
VS_T_ID_HT       0
VS_T_ID_KT       0
VS_T_ID_LG       0
VS_T_ID_LT       0
VS_T_ID_NC       0
VS_T_ID_OB       0
VS_T_ID_SK       0
VS_T_ID_SS       0
VS_T_ID_WO       0
HOME             0
AWAY             0
inplay_ratio    77
P_ID             0
dtype: int64

In [31]:
df1 = df1.fillna(0)

In [32]:
df1['P_ID']=df1['P_ID'].astype('int')

In [33]:
batter_left

,year,month,P_ID,PA,T_ID,BABIP,BABIP_2020,total_BABIP
0,2020,10,65304,3.0,WO,0.375000,1.000000,0.875000
1,2020,10,64012,10.0,KT,0.297619,0.227273,0.241342
2,2020,10,68204,4.0,OB,0.315789,0.315789,0.315789
3,2020,10,66203,6.0,OB,0.285714,0.352941,0.339496
4,2020,10,50293,2.0,OB,0.266667,0.266667,0.266667
...,...,...,...,...,...,...,...,...
222,2020,10,63435,4.0,SS,0.166667,0.166667,0.166667
223,2020,10,69418,0.0,SS,0.299363,0.300699,0.300432
224,2020,10,61457,0.0,SS,0.000000,0.000000,0.000000
225,2020,10,66707,7.0,HH,0.272727,0.285714,0.283117


In [34]:
data = batter_tidy.loc[(batter_tidy['year']==2020) & (batter_tidy['month']==9)]

In [35]:
data

,year,month,P_ID,T_ID,AB,HIT,H2,H3,HR,BB,...,VS_T_ID_KT,VS_T_ID_LG,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,HOME,AWAY
7197,2020,9,65304,WO,2,2,0,0,0,0,...,2,2,2,2,5,4,2,0,14,13
7198,2020,9,64012,KT,7,4,1,0,1,2,...,0,2,3,2,4,4,4,2,13,12
7199,2020,9,68204,OB,4,2,1,0,0,0,...,4,2,0,2,0,2,4,5,14,11
7200,2020,9,66203,OB,6,3,1,0,0,0,...,4,2,0,2,0,2,4,5,14,11
7201,2020,9,50293,OB,2,1,0,0,0,0,...,4,2,0,2,0,2,4,5,14,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7419,2020,9,63435,SS,4,0,0,0,0,0,...,4,2,2,3,4,2,0,2,13,11
7420,2020,9,69418,SS,0,0,0,0,0,0,...,4,2,2,3,4,2,0,2,13,11
7421,2020,9,61457,SS,0,0,0,0,0,0,...,4,2,2,3,4,2,0,2,13,11
7422,2020,9,66707,HH,7,0,0,0,0,0,...,2,2,2,2,5,2,2,4,16,9


In [36]:
p_id = data['P_ID'].reset_index(drop = True)

In [37]:
batter_2020 = pd.DataFrame()

for i in range(len(p_id)):
    pid = p_id[i]
    batter = df1.loc[df1.P_ID == pid,:]
    batter_2020 = batter_2020.append(batter)
    

In [38]:
batter_2020 = batter_2020.reset_index(drop = True)

In [39]:
batter_2020

,year,month,AB,HIT,H2,H3,HR,BB,HP,KK,...,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,HOME,AWAY,inplay_ratio,P_ID
0,4040,17,2,2,0,0,0,0,0,0,...,7,4,5,4,6,0,29,24,0.666667,65304
1,10100,35,33,6,1,0,1,4,0,12,...,12,15,12,12,13,13,57,58,0.550000,64012
2,8080,30,47,12,1,0,0,3,0,13,...,8,13,0,9,13,11,50,45,0.703704,68204
3,6060,23,22,6,2,0,0,2,0,6,...,6,12,0,6,13,6,35,37,0.680000,66203
4,4040,15,17,4,0,0,0,3,1,2,...,0,8,0,4,7,6,23,25,0.714286,50293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,4040,14,8,1,0,0,0,0,0,2,...,5,9,7,2,0,5,28,20,0.750000,63435
223,10100,35,168,44,11,0,1,7,2,28,...,13,10,16,12,0,16,62,56,0.790055,69418
224,2020,9,0,0,0,0,0,0,0,0,...,2,3,4,2,0,2,13,11,0.000000,61457
225,8080,30,34,6,2,0,0,0,1,13,...,10,12,10,10,11,10,50,44,0.600000,66707


In [40]:
batter_left= pd.merge(batter_left, df1[['P_ID','inplay_ratio']], left_on='P_ID', right_on='P_ID', how='left')

In [41]:
batter_left.index = batter_left['P_ID']
df1 = df1.reset_index(drop = True)

In [42]:
batter_left = batter_left.reset_index(drop = True)

In [43]:
batter_2020['inplay'] = batter_2020['AB'] - batter_2020['KK'] - batter_2020['HR'] + batter_2020['SF']

In [44]:
batter_2020['inplay_hit'] = batter_2020['HIT'] - batter_2020['HR']

In [45]:
batter_left= pd.merge(batter_left, batter_2020[['P_ID', 'inplay', 'inplay_hit']], left_on='P_ID', right_on='P_ID', how='left')

In [46]:
batter_left

,year,month,P_ID,PA,T_ID,BABIP,BABIP_2020,total_BABIP,inplay_ratio,inplay,inplay_hit
0,2020,10,65304,3.0,WO,0.375000,1.000000,0.875000,0.666667,2.0,2
1,2020,10,64012,10.0,KT,0.297619,0.227273,0.241342,0.550000,22.0,5
2,2020,10,68204,4.0,OB,0.315789,0.315789,0.315789,0.703704,38.0,12
3,2020,10,66203,6.0,OB,0.285714,0.352941,0.339496,0.680000,17.0,6
4,2020,10,50293,2.0,OB,0.266667,0.266667,0.266667,0.714286,15.0,4
...,...,...,...,...,...,...,...,...,...,...,...
222,2020,10,63435,4.0,SS,0.166667,0.166667,0.166667,0.750000,6.0,1
223,2020,10,69418,0.0,SS,0.299363,0.300699,0.300432,0.790055,143.0,43
224,2020,10,61457,0.0,SS,0.000000,0.000000,0.000000,0.000000,0.0,0
225,2020,10,66707,7.0,HH,0.272727,0.285714,0.283117,0.600000,21.0,6


In [47]:
batter_left['pred_inplay'] = batter_left['PA'] * batter_left['inplay_ratio']

In [48]:
batter_left

,year,month,P_ID,PA,T_ID,BABIP,BABIP_2020,total_BABIP,inplay_ratio,inplay,inplay_hit,pred_inplay
0,2020,10,65304,3.0,WO,0.375000,1.000000,0.875000,0.666667,2.0,2,2.000000
1,2020,10,64012,10.0,KT,0.297619,0.227273,0.241342,0.550000,22.0,5,5.500000
2,2020,10,68204,4.0,OB,0.315789,0.315789,0.315789,0.703704,38.0,12,2.814815
3,2020,10,66203,6.0,OB,0.285714,0.352941,0.339496,0.680000,17.0,6,4.080000
4,2020,10,50293,2.0,OB,0.266667,0.266667,0.266667,0.714286,15.0,4,1.428571
...,...,...,...,...,...,...,...,...,...,...,...,...
222,2020,10,63435,4.0,SS,0.166667,0.166667,0.166667,0.750000,6.0,1,3.000000
223,2020,10,69418,0.0,SS,0.299363,0.300699,0.300432,0.790055,143.0,43,0.000000
224,2020,10,61457,0.0,SS,0.000000,0.000000,0.000000,0.000000,0.0,0,0.000000
225,2020,10,66707,7.0,HH,0.272727,0.285714,0.283117,0.600000,21.0,6,4.200000


In [49]:
batter_left['pred_inplay_hit'] = batter_left['total_BABIP'] * (batter_left['inplay'] + batter_left['pred_inplay']) - batter_left['inplay_hit']

In [50]:
batter_left = batter_left[['year', 'month', 'P_ID', 'T_ID', 'PA', 'pred_inplay_hit']]

In [51]:
batter_left = batter_left.fillna(0)

In [52]:
batter_left.pred_inplay_hit[batter_left.pred_inplay_hit < 0.0 ]=0

C:\Users\dhxog\anaconda3\envs\my\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [53]:
batter_left

,year,month,P_ID,T_ID,PA,pred_inplay_hit
0,2020,10,65304,WO,3.0,1.500000
1,2020,10,64012,KT,10.0,1.636905
2,2020,10,68204,OB,4.0,0.888889
3,2020,10,66203,OB,6.0,1.156571
4,2020,10,50293,OB,2.0,0.380952
...,...,...,...,...,...,...
222,2020,10,63435,SS,4.0,0.500000
223,2020,10,69418,SS,0.0,0.000000
224,2020,10,61457,SS,0.0,0.000000
225,2020,10,66707,HH,7.0,1.134545


In [54]:
batter_2020_ratio = pd.read_csv("https://raw.githubusercontent.com/njj06135/Baseball_ChilliShrimp/master/data/batter_2020_ratio.csv")

In [55]:
batter_2020_ratio

,P_ID,H2_rate,H3_rate,HR_rate,BB_rate,HP_rate,KK_rate,SB_try,SB_succ,GD_rate,SF_rate
0,50054,0.230769,0.000000,0.000000,0.095890,0.013699,0.178082,0.027397,0.500000,0.000000,0.000000
1,50066,0.333333,0.000000,0.166667,0.074074,0.000000,0.222222,0.000000,0.000000,0.037037,0.000000
2,50150,0.200000,0.000000,0.000000,0.033333,0.000000,0.200000,0.166667,1.000000,0.066667,0.000000
3,50165,0.149123,0.017544,0.298246,0.109442,0.006438,0.274678,0.004292,1.000000,0.012876,0.008584
4,50202,0.000000,0.000000,0.000000,0.033333,0.000000,0.133333,0.033333,1.000000,0.033333,0.033333
...,...,...,...,...,...,...,...,...,...,...,...
349,79456,0.132353,0.029412,0.000000,0.060000,0.016000,0.100000,0.008000,0.500000,0.032000,0.016000
350,79545,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
351,79608,0.245283,0.000000,0.056604,0.076190,0.016667,0.104762,0.040476,0.823529,0.021429,0.014286
352,79705,0.333333,0.000000,0.000000,0.120000,0.040000,0.240000,0.000000,0.000000,0.080000,0.120000


In [56]:
batter_left = pd.merge(batter_left, batter_2020_ratio, left_on='P_ID', right_on='P_ID', how='left')

In [57]:
batter_left

,year,month,P_ID,T_ID,PA,pred_inplay_hit,H2_rate,H3_rate,HR_rate,BB_rate,HP_rate,KK_rate,SB_try,SB_succ,GD_rate,SF_rate
0,2020,10,65304,WO,3.0,1.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2020,10,64012,KT,10.0,1.636905,0.166667,0.0,0.166667,0.100000,0.000000,0.300000,0.075000,0.666667,0.000000,0.050000
2,2020,10,68204,OB,4.0,0.888889,0.083333,0.0,0.000000,0.055556,0.000000,0.240741,0.018519,1.000000,0.000000,0.074074
3,2020,10,66203,OB,6.0,1.156571,0.333333,0.0,0.000000,0.080000,0.000000,0.240000,0.000000,0.000000,0.000000,0.040000
4,2020,10,50293,OB,2.0,0.380952,0.000000,0.0,0.000000,0.142857,0.047619,0.095238,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,2020,10,63435,SS,4.0,0.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.250000,0.125000,1.000000,0.000000,0.000000
223,2020,10,69418,SS,0.0,0.000000,0.250000,0.0,0.022727,0.038674,0.011050,0.154696,0.049724,0.444444,0.022099,0.022099
224,2020,10,61457,SS,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
225,2020,10,66707,HH,7.0,1.134545,0.333333,0.0,0.000000,0.000000,0.028571,0.371429,0.000000,0.000000,0.028571,0.000000


In [58]:
batter_left['HR'] = (batter_left['HR_rate']*batter_left['pred_inplay_hit'])/(1-batter_left['HR_rate'])

In [59]:
batter_left['HIT'] = batter_left['HR'] + batter_left['pred_inplay_hit']

In [60]:
batter_left['H2'] = batter_left['HIT']*batter_left['H2_rate']
batter_left['H3'] = batter_left['HIT']*batter_left['H3_rate']

In [61]:
batter_left = batter_left.fillna(0)

In [62]:
batter_left['BB'] = batter_left['PA'] * batter_left['BB_rate']
batter_left['HP'] = batter_left['PA'] * batter_left['HP_rate']
batter_left['SF'] = batter_left['PA'] * batter_left['SF_rate']
batter_left['AB'] = batter_left['PA'] - batter_left['BB'] - batter_left['HP'] - batter_left['SF']

In [63]:
batter_left['AVG'] = batter_left['HIT'] / batter_left['AB']
batter_left['OBP'] = (batter_left['HIT'] + batter_left['BB'] + batter_left['HP']) / batter_left['PA']
batter_left['SLG'] = ((batter_left['HIT'] - batter_left['H2'] - batter_left['H3'] - batter_left['HR']) + 2 * batter_left['H2'] + 3 * batter_left['H3'] + 4 * batter_left['HR']) / batter_left['AB']
batter_left['OPS'] = batter_left['OBP'] + batter_left['SLG']

In [66]:
batter_feature = pd.read_csv("https://raw.githubusercontent.com/njj06135/Baseball_ChilliShrimp/master/data/batter_feature_selection.csv")

In [68]:
batter_feature = batter_feature[['year', 'month', 'P_ID', 'BAT_ORDER']]

In [77]:
batter_feature = batter_feature.loc[batter_feature.year == 2020 ,:]
batter_feature = batter_feature.loc[batter_feature.month == 9,:]

batter_feature = batter_feature[[ 'P_ID', 'BAT_ORDER' ]]

In [80]:
batter_left = pd.merge(batter_left, batter_feature, left_on='P_ID', right_on='P_ID', how='left')

In [84]:
batter_left['KK'] = batter_left['PA'] * batter_left['KK_rate']

batter_left['BABIP'] = (batter_left['HIT'] - batter_left['HR']) / (batter_left['AB'] - batter_left['HR']- batter_left['KK'] + batter_left['SF']) 

In [86]:
batter_left = batter_left[['year', 'month', 'P_ID', 'T_ID', 'PA', 'BAT_ORDER', 'SB_try', 'SB_succ', 'BABIP', 'H2_rate', 'H3_rate', 'HR_rate', 'BB_rate', 'HP_rate', 'GD_rate', 'AVG', 'OBP', 'SLG', 'OPS']]

In [87]:
batter_left = batter_left.fillna(0)

In [88]:
schedule = pd.read_csv("https://raw.githubusercontent.com/njj06135/Baseball_ChilliShrimp/master/data/schedule.csv", encoding='CP949')

In [89]:
schedule = schedule.loc[(schedule['year']==2020) & (schedule['month']==10)]

In [90]:
batter_left = pd.merge(batter_left, schedule, left_on='T_ID', right_on='T_ID', how='left')

In [91]:
batter_left.to_csv('batter_for_pred.csv')